# Low-Level Examples 
Here we present low-level examples of integrating LM-Polygraph into LLM inference using HF library.

## Initialize

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
model_name_or_path = "mistralai/Mistral-7B-Instruct-v0.2"
device = "cuda:0"
dataset_name = "LM-Polygraph/triviaqa"
batch_size = 2

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True
)

model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    quantization_config=quantization_config,
    device_map=device,
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

## Sequence-Level Examples

In [4]:
messages = [
    [
        {
            "role": "user", 
            "content": "How many fingers on a coala's foot?"
        }
    ],
    [
        {
            "role": "user",
            "content": "Who sang a song Yesterday?"
        }
    ],
    [
        {
            "role": "user",
            "content": "Кто спел песню Кукла Колдуна?"
        }
    ],
    [
        {
            "role": "user",
            "content": "Translate into French: 'I want a small cup of coffee'"
        }
    ]
]

chat_messages = [tokenizer.apply_chat_template(m, tokenize=False) for m in messages]

In [5]:
from lm_polygraph.stat_calculators import GreedyProbsCalculator, GreedyAlternativesNLICalculator
from lm_polygraph.estimators.claim_conditioned_probability import ClaimConditionedProbability
from lm_polygraph.utils.deberta import Deberta
from lm_polygraph.utils.generation_parameters import GenerationParameters
from lm_polygraph.model_adapters import WhiteboxModel


max_new_tokens = 100
generation_params = GenerationParameters()
generation_params.temperature = 0.9
generation_params.do_sample = True

model_adapter = WhiteboxModel(model, tokenizer, generation_parameters=generation_params)

calc_greedy_probs = GreedyProbsCalculator()
nli_model = Deberta(device=device)
nli_model.setup()
calc_nli = GreedyAlternativesNLICalculator(nli_model=nli_model)

estimator = ClaimConditionedProbability()

Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
from torch.utils.data import DataLoader


data_loader = DataLoader(chat_messages, batch_size=batch_size, shuffle=False, collate_fn=lambda x: x)
for batch in data_loader:
    deps = dict()
    deps.update(calc_greedy_probs(
        deps, texts=batch, model=model_adapter, max_new_tokens=max_new_tokens))
    deps.update(calc_nli(deps, texts=None, model=model_adapter))

    uncertainty_scores = estimator(deps)
    generated_texts = tokenizer.batch_decode(deps['greedy_tokens'])
    
    for text, ue_score in zip(generated_texts, uncertainty_scores):
        print("Output:", text)
        print("Uncertainty score:", ue_score)
        print()

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


/home/artemshelmanov/conda/lib/python3.12/site-packages/bitsandbytes/nn/modules.py:463: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
`torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True`. Falling back to eager attention. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Output: A koala's paws do not have an opposable thumb or any fingers like humans do. Instead, they have five non-opposable, roughly similar digits on each of their forepaws and four on their hind paws, for a total of 20 digits on both their front and hind paws. However, since fingers in the human sense do not exist on a koala's paws, it's not an accurate question to ask.</s>
Uncertainty score: -1.2142750309824153e-06

Output: The song "Yesterday" was written and performed by Paul McCartney, but credit is often given to The Beatles as a group since the song was released under their name. However, McCartney performs the solo version on the recording, and the quartet only provided the instrumental background on the original 1965 release. To clarify, Paul McCartney sang "Yesterday."</s>
Uncertainty score: -3.7164147931175543e-07

Output: I'm an artificial intelligence and don't have the ability to sing or listen to music. However, I can help you find information about the performer of the 

## Claim-Level Examples
Here we split text into actomic claims and quantify uncertainty of individual claims

In [7]:
messages = [
    [
        {
            "role": "user", 
            "content": "Tell me a bio of Albert Einstein."
        }
    ],
    [
        {
            "role": "user",
            "content": "Tell me a bio of Alla Pugacheva."
        }
    ],
    [
        {
            "role": "user",
            "content": "Tell me a bio of Paul McCartney."
        }
    ]
]

chat_messages = [tokenizer.apply_chat_template(m, tokenize=False) for m in messages]

In [ ]:
import os

from lm_polygraph.model_adapters import WhiteboxModelBasic
from lm_polygraph.estimators import ClaimConditionedProbabilityClaim
from lm_polygraph.stat_calculators import *
from lm_polygraph.utils.openai_chat import OpenAIChat
from lm_polygraph.utils.deberta import Deberta


max_new_tokens = 100
generation_params = GenerationParameters()
generation_params.temperature = 0.9
generation_params.do_sample = True

model_adapter = WhiteboxModel(model, tokenizer, generation_parameters=generation_params)

calc_greedy_probs = GreedyProbsCalculator()
os.environ["OPENAI_API_KEY"] = "<Your OpenAI API key>"
calc_claim_extractor = ClaimsExtractor(OpenAIChat("gpt-4o"))
calc_claim_nli = GreedyAlternativesNLICalculator(Deberta(device=device))

estimator = ClaimConditionedProbabilityClaim()

Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
from torch.utils.data import DataLoader


data_loader = DataLoader(chat_messages, batch_size=batch_size, shuffle=False, collate_fn=lambda x: x)
for batch in data_loader:
    encoded = tokenizer(batch, padding=True, return_tensors="pt").to(device)

    deps = {"model_inputs": encoded}
    deps.update(calc_greedy_probs(
        deps, texts=batch, model=model_adapter, max_new_tokens=max_new_tokens))
    deps.update({"greedy_texts" : tokenizer.batch_decode(deps['greedy_tokens'])})
    deps.update(calc_claim_extractor(deps, texts=batch, model=model_adapter))
    deps.update(calc_claim_nli(deps, texts=None, model=model_adapter))

    uncertainty_scores = estimator(deps)

    for text, claims, ue_score in zip(deps["greedy_texts"], deps['claims'], uncertainty_scores):
        print("Output:", text)
        
        for claim, ue in zip(claims, ue_score):
            print("claim:", claim.claim_text)
            print("aligned tokens:", claim.aligned_token_ids)
            print("UE score:", ue)

        print()

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Output: Albert Einstein (March 14, 1879 – April 18, 1955) was a German-born theoretical physicist who is widely regarded as one of the most influential scientists in history. He is best known for developing the theory of general relativity, one of the two pillars of modern physics (alongside quantum mechanics). His work is also known for its influence on the philosophy of science.

Einstein was born in
claim: Albert Einstein was born on March 14, 1879.
aligned tokens: [0, 1, 3, 4, 6, 7, 10, 11, 12, 13]
UE score: -0.7957917064115393
claim: Albert Einstein died on April 18, 1955.
aligned tokens: [0, 1, 15, 17, 18, 21, 22, 23, 24]
UE score: -0.9997172893343415
claim: Albert Einstein was a German-born theoretical physicist.
aligned tokens: [0, 1, 26, 27, 28, 29, 30, 31, 32, 33, 34]
UE score: -0.45843554806918824
claim: Albert Einstein is widely regarded as one of the most influential scientists in history.
aligned tokens: [0, 1, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]
UE score: -0.